In [ ]:
# import your libraries
import tweepy
import json
import requests
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
# open file with twitter keys (only need bearer token for this example)
credentials = "keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [ ]:
bearer_token = api_tokens["bearer_token"]

In [ ]:
# create the client object which will allow you to interact with the Twitter API
client = tweepy.Client(bearer_token=bearer_token)

In [ ]:
duolingo_data = client.search_recent_tweets(
    # building a query string: https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query
    query = "#Duolingo365 -is:retweet lang:en",
    max_results = 100
)

duolingo_data.data

In [ ]:
# create a list to only store the tweets, none of the metadata
tweets_duolingo = []

for tweet in duolingo_data.data:
    tweets_duolingo.append(tweet.text)
    
tweets_duolingo

In [ ]:
morgan_data = client.search_recent_tweets(
    query="Piers Morgan -is:retweet lang:en",
    max_results = 100
)

morgan_data.data

In [ ]:
morgan_tweets = []
for tweet in morgan_data.data:
    morgan_tweets.append(tweet.text)
    
morgan_tweets

In [ ]:
import random

# create a function to format our tweet into a collection of key-value pairs (ie features)
def setup_features(text):
    # create array of keywords
    # make text lowercase
    text = text.lower()
    # split the text into an array of words
    words_list = text.split()
    # this isn't nessecary but will make the next part shorter
    s_words = stopwords.words('english')
    keywords = [ term for term in words_list if term not in s_words and 'http' not in term] 
    random.shuffle(keywords)
    while len(keywords) < 3:
        keywords.append('None')
    return {'first_keyword': keywords[0], 'second_keyword': keywords[1], 'third_keyword': keywords[2]}

In [ ]:
# create a list of tuples containing the featureset and the label
# features and label list will contain my features and my label
features_and_labels = []
for tweet in tweets_duolingo:
    features_and_labels.append(  (setup_features(tweet), 'Duolingo')    )

In [ ]:
for tweet in morgan_tweets:
    features_and_labels.append(  (setup_features(tweet), 'Piers Morgan')    )

In [ ]:
# print out our features and labels
features_and_labels

In [ ]:
# ALWAYS SHUFFLE YOUR FEATURES AND LABELS!!!!
random.shuffle(features_and_labels)

In [ ]:
# print out our shuffled features and labels
features_and_labels

In [ ]:
# split 80:20 training data and testing data

import math

# get how many elements in my list is 80%
split_num = math.floor(len(features_and_labels) * .8)
split_num

In [ ]:
# 80% of my data
training_set = features_and_labels[:split_num]
# 20% of my data
testing_set = features_and_labels[split_num:]

In [ ]:
import nltk
# create our classifier

# My baseline is 50% because I have two labels/outcomes - we are hoping to do better than the baseline!
my_classifier = nltk.NaiveBayesClassifier.train(training_set)

In [ ]:
# grab a random tweet from Twitter and see if it is classified correctly
my_classifier.classify(setup_features("Big Piers Morgan getting told off by specsavers has made my day."))

In [ ]:
# test our accuracy
nltk.classify.accuracy(my_classifier, testing_set)

In [ ]:
classifier.show_most_informative_features(10)